## Direct Prediction by other spatial temporal methods

The part is mainly implemented with the library OpenSTL(). To run this notebook, you need to install OpenSTL first. My experience of the installation is that set the version of **timm==0.9**, this can avoid plenty errors. 

The models tried here are ConvLSTM and SimVp. Using the template config for mmnist. To keep accordance with the FNO experiments, the optimizer is set to Adam, and the epochs is set to 500.  
* ConvLSTM
* SimVp-gsta 

In [1]:
import numpy as np
import os
import torch 
from torch.utils.data import Dataset
from utilities3 import *
from openstl.api import BaseExperiment
from openstl.utils import create_parser, default_parser

os.environ['CUDA_VISIBLE_DEVICES']='0'

/home/zhengyaokun/anaconda3/envs/OpenSTL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read and split the data
def read_data(file_paths, inlen=16, outlen=16):
    eta = []
    scatter = []
    len = inlen + outlen
    for f in file_paths:
        reader = MatReader(f, to_torch=False)
        eta_tmp = reader.read_field('eta')
        scatter_tmp = reader.read_field('scatter')
        #eta_tmp = eta_tmp[:,:,:,117:501]
        #scatter_tmp = scatter_tmp[:,:,:,117:501]
        eta_tmp = eta_tmp[:,:,:,117:437]
        scatter_tmp = scatter_tmp[:,:,:,117:437]
        #print(eta_tmp.shape,scatter_tmp.shape)
        slice_eta = []
        slice_scatter = []
        for i in range(10):
            slice_eta.append(eta_tmp[:,:,:,32*i:32*(i+1)])
            slice_scatter.append(scatter_tmp[:,:,:,32*i:32*(i+1)])
        eta_tmp = np.concatenate(slice_eta,axis=0)
        scatter_tmp = np.concatenate(slice_scatter,axis=0)
        #print(eta_tmp.shape,scatter_tmp.shape)
        #eta.append(eta_tmp)
        #scatter.append(scatter_tmp)
        x = reader.read_field('x').flatten()
        y = reader.read_field('y').flatten()
        t = reader.read_field('t').flatten()
        t = t[0:32] 
        if eta == []:
            eta = eta_tmp
            scatter = scatter_tmp
        else:
            eta = np.concatenate([eta,eta_tmp])
            scatter = np.concatenate([scatter,scatter_tmp])
    return eta, scatter, x, y, t

def split_data(eta, scatter, train_test_split=[0.6,0.2,0.2], inlen=16, outlen=16, sub=2, to_torch=False):
    train_size = int(eta.shape[0]*train_test_split[0])
    test_size = int(eta.shape[0]*train_test_split[2])
    train_a = scatter[:train_size,::sub,::sub,:inlen]
    train_e = eta[:train_size,::sub,::sub,:inlen]
    train_u = eta[:train_size,::sub,::sub,inlen:inlen+outlen]
    val_a = scatter[train_size:train_size+test_size,::sub,::sub,:inlen]
    val_e = eta[train_size:train_size+test_size,::sub,::sub,:inlen]
    val_u = eta[train_size:train_size+test_size,::sub,::sub,inlen:inlen+outlen]
    test_a = scatter[train_size+test_size:,::sub,::sub,:inlen]
    test_e = eta[train_size+test_size:,::sub,::sub,:inlen]
    test_u = eta[train_size+test_size:,::sub,::sub,inlen:inlen+outlen]
    if to_torch:
        train_a = torch.from_numpy(train_a)
        train_e = torch.from_numpy(train_e)
        train_u = torch.from_numpy(train_u)
        val_a = torch.from_numpy(val_a)
        val_e = torch.from_numpy(val_e)
        val_u = torch.from_numpy(val_u)
        test_a = torch.from_numpy(test_a)
        test_e = torch.from_numpy(test_e)
        test_u = torch.from_numpy(test_u)
    return train_a, train_e, train_u, val_a, val_e, val_u, test_a, test_e, test_u

# read and split data, for each datafile, spilit it in to train,val and test then concatenate them, add hs and spread tag in the filename, length the same as the test array
def read_and_split(file_paths, train_test_split=[0.6,0.2,0.2], inlen=16, outlen=16, sub=2, to_torch=False):
    eta, scatter, x, y, t = read_data([file_paths[0]], inlen=inlen, outlen=outlen)
    train_a, train_e, train_u, val_a, val_e, val_u, test_a, test_e, test_u = split_data(eta, scatter, train_test_split=train_test_split, inlen=inlen, outlen=outlen, sub=sub, to_torch=to_torch)
    hs = float(file_paths[0].split('angle')[1].split('h')[1].split('.mat')[0]) * np.ones(test_a.shape[0])   
    spread = float(file_paths[0].split('angle')[1].split('h')[0]) * np.ones(test_a.shape[0]) 

    for i,f in enumerate(file_paths):
        if i>0:
            eta, scatter, x, y, t = read_data([f], inlen=inlen, outlen=outlen)
            train_a_tmp, train_e_tmp, train_u_tmp, val_a_tmp, val_e_tmp, val_u_tmp, test_a_tmp, test_e_tmp, test_u_tmp = split_data(eta, scatter, train_test_split=train_test_split, inlen=inlen, outlen=outlen, sub=sub, to_torch=to_torch)
            train_a = np.concatenate([train_a,train_a_tmp])
            train_e = np.concatenate([train_e,train_e_tmp])
            train_u = np.concatenate([train_u,train_u_tmp])
            val_a = np.concatenate([val_a,val_a_tmp])
            val_e = np.concatenate([val_e,val_e_tmp])
            val_u = np.concatenate([val_u,val_u_tmp])
            test_a = np.concatenate([test_a,test_a_tmp])
            test_e = np.concatenate([test_e,test_e_tmp])
            test_u = np.concatenate([test_u,test_u_tmp])
            hs = np.concatenate([hs,float(f.split('angle')[1].split('h')[1].split('.mat')[0]) * np.ones(test_a_tmp.shape[0])])
            spread = np.concatenate([spread,float(f.split('angle')[1].split('h')[0]) * np.ones(test_a_tmp.shape[0])])
    if to_torch:
        train_a = torch.from_numpy(train_a)
        train_e = torch.from_numpy(train_e)
        train_u = torch.from_numpy(train_u)
        val_a = torch.from_numpy(val_a)
        val_e = torch.from_numpy(val_e)
        val_u = torch.from_numpy(val_u)
        test_a = torch.from_numpy(test_a)
        test_e = torch.from_numpy(test_e)
        test_u = torch.from_numpy(test_u)
        hs = torch.from_numpy(hs)
        spread = torch.from_numpy(spread)

    return train_a, train_e, train_u, val_a, val_e, val_u, test_a, test_e, test_u, x, y, t, hs, spread


dirs = os.listdir("../mixed_data")
dirs = [os.path.join("../mixed_data",d) for d in dirs]
# add another dir
dirs2 = os.listdir("../mixed_data2")
dirs2 = [os.path.join("../mixed_data2",d) for d in dirs2]
dirs = dirs + dirs2
train_a, train_e, train_u, val_a, val_e, val_u, test_a, test_e, test_u,x_domain,y_domain,t_domain,hs, spread = read_and_split(dirs,inlen=16,outlen=16,sub=2,to_torch=False)
hs = hs.reshape(-1,1)

In [3]:
# Custom dataset and build the dataloader
class CustomDataset(Dataset):
    def __init__(self, X, Y, normalize=False):
        super(CustomDataset, self).__init__()
        self.X = X
        self.Y = Y
        self.mean = None
        self.std = None
        self.data_name = "custom"

        if normalize:
            # get the mean/std values along the channel dimension
            mean = data.mean(axis=(0, 1, 2, 3)).reshape(1, 1, -1, 1, 1)
            std = data.std(axis=(0, 1, 2, 3)).reshape(1, 1, -1, 1, 1)
            data = (data - mean) / std
            self.mean = mean
            self.std = std

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        data = torch.tensor(self.X[index]).float()
        labels = torch.tensor(self.Y[index]).float()
        return data, labels

batch_size = 20

train_a = np.transpose(np.expand_dims(train_a,axis=1),axes=(0,4,1,2,3))
train_u = np.transpose(np.expand_dims(train_u,axis=1),axes=(0,4,1,2,3))
val_a = np.transpose(np.expand_dims(val_a,axis=1),axes=(0,4,1,2,3))
val_u = np.transpose(np.expand_dims(val_u,axis=1),axes=(0,4,1,2,3))
test_a = np.transpose(np.expand_dims(test_a,axis=1),axes=(0,4,1,2,3))
test_u = np.transpose(np.expand_dims(test_u,axis=1),axes=(0,4,1,2,3))

train_set = CustomDataset(X=train_a, Y=train_u)
val_set = CustomDataset(X=val_a, Y=val_u)
test_set = CustomDataset(X=test_a, Y=test_u)
dataloader_train = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_val = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, shuffle=False, pin_memory=True)
dataloader_test = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
# config for SimVP-gSTA, setting copy from './OpenSTL/configs/mmnist/simvp/SimVP_gSTA.py'
pre_seq_length = 16
aft_seq_length = 16
custom_training_config = {
    'pre_seq_length': pre_seq_length,
    'aft_seq_length': aft_seq_length,
    'total_length': pre_seq_length + aft_seq_length,
    'batch_size': batch_size,
    'val_batch_size': batch_size,
    'epoch': 500,
    'lr': 0.001,   
    'metrics': ['mse', 'mae'],
    'dataname': 'custom',
    'ex_name': 'simvp',
    'in_shape': [16, 1, 64, 64],
    'optimizer': 'adam',
}

custom_model_config = {
    # For MetaVP models, the most important hyperparameters are: 
    # N_S, N_T, hid_S, hid_T, model_type
    #'method': 'SimVP',
    # Users can either using a config file or directly set these hyperparameters 
    #'config_file': './OpenSTL/configs/mmnist/simvp/SimVP_gSTA-L.py',
    # Here, we directly set these parameters
    'method': 'SimVP',
    # model
    'spatio_kernel_enc': 3,
    'spatio_kernel_dec': 3,
    'model_type': 'gSTA',
    'hid_S': 64,
    'hid_T': 512,
    'N_T': 8,
    'N_S': 4,
    # training
    'lr': 1e-3,
    #'batch_size': 16,
    'drop_path': 0,
    'sched': 'onecycle',
}

In [10]:
# config for ConvLSTM, seting copy from  './OpenSTL/configs/mmnist/ConvLSTM.py'
pre_seq_length = 16
aft_seq_length = 16
custom_training_config = {
    'pre_seq_length': pre_seq_length,
    'aft_seq_length': aft_seq_length,
    'total_length': pre_seq_length + aft_seq_length,
    'batch_size': batch_size,
    'val_batch_size': batch_size,
    'epoch': 500,
    'lr': 0.001,   
    'metrics': ['mse', 'mae'],
    'dataname': 'custom',
    'ex_name': 'convlstm',
    'in_shape': [16, 1, 64, 64],
    'optimizer': 'adam',
}

custom_model_config = {
    # For MetaVP models, the most important hyperparameters are: 
    # N_S, N_T, hid_S, hid_T, model_type
    #'method': 'SimVP',
    # Users can either using a config file or directly set these hyperparameters 
    #'config_file': './OpenSTL/configs/mmnist/ConvLSTM-L.py',
    # Here, we directly set these parameters
    'method': 'ConvLSTM',
# reverse scheduled sampling
    'reverse_scheduled_sampling': 0,
    'r_sampling_step_1': 25000,
    'r_sampling_step_2': 50000,
    'r_exp_alpha': 5000,
    # scheduled sampling
    'scheduled_sampling': 1,
    'sampling_stop_iter': 50000,
    'sampling_start_value': 1.0,
    'sampling_changing_rate': 0.00002,
    # model
    'num_hidden': '128,128,128,128',
    'filter_size': 5,
    'stride': 1,
    'patch_size': 4,
    'layer_norm': 0,
    # training
    'lr': 5e-4,
    #'batch_size': 16,
    'sched': 'onecycle',
}


In [11]:
# set up the experiment
args = create_parser().parse_args([])
config = args.__dict__

# update the training config
config.update(custom_training_config)
# update the model config
config.update(custom_model_config)
# fulfill with default values
default_values = default_parser()
for attribute in default_values.keys():
    if config[attribute] is None:
        config[attribute] = default_values[attribute]

exp = BaseExperiment(args, dataloaders=(dataloader_train, dataloader_val, dataloader_test))


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
print(config)

{'device': 'cuda', 'dist': False, 'res_dir': 'work_dirs', 'ex_name': 'convlstm', 'fp16': False, 'torchscript': False, 'seed': 42, 'fps': False, 'test': False, 'deterministic': False, 'batch_size': 20, 'val_batch_size': 20, 'num_workers': 4, 'data_root': './data', 'dataname': 'custom', 'pre_seq_length': 16, 'aft_seq_length': 16, 'total_length': 32, 'use_augment': False, 'use_prefetcher': False, 'drop_last': False, 'method': 'convlstm', 'config_file': None, 'model_type': 'gSTA', 'drop': 0.0, 'drop_path': 0.0, 'overwrite': False, 'epoch': 500, 'log_step': 1, 'opt': 'adam', 'opt_eps': None, 'opt_betas': None, 'momentum': 0.9, 'weight_decay': 0.0, 'clip_grad': None, 'clip_mode': 'norm', 'no_display_method_info': False, 'sched': 'onecycle', 'lr': 0.0005, 'lr_k_decay': 1.0, 'warmup_lr': 1e-05, 'min_lr': 1e-06, 'final_div_factor': 10000.0, 'warmup_epoch': 0, 'decay_epoch': 100, 'decay_rate': 0.1, 'filter_bias_and_bn': False, 'gpus': [0], 'metric_for_bestckpt': 'val_loss', 'ckpt_path': None, 'm

In [13]:
# train and test
print('>'*35 + ' training ' + '<'*35)
exp.train()

print('>'*35 + ' testing  ' + '<'*35)
exp.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params
---------------------------------------------
0 | model     | ConvLSTM_Model | 15.1 M
1 | criterion | MSELoss        | 0     
---------------------------------------------
15.1 M    Trainable params
0         Non-trainable params
15.1 M    Total params
60.334    Total estimated model params size (MB)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> training <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:26:04) [GCC 10.4.0]
CUDA available: True
CUDA_HOME: /usr/local/cuda-11.2
NVCC: Build cuda_11.2.r11.2/compiler.29373293_0
GPU 0: NVIDIA A100 80GB PCIe
GCC: gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
PyTorch: 2.1.0+cu121
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.2-Product Build 20230613 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_6

Epoch 0: 100%|██████████| 36/36 [00:09<00:00,  3.83it/s, v_num=9, train_loss_step=1.24e+3, train_loss_epoch=1.1e+3]

Epoch 0, global step 36: 'val_loss' reached 1.95434 (best 1.95434), saving model to '/home/zhengyaokun/radar_data_generation/radarWave/fno_hosdata/othermethod/work_dirs/convlstm/checkpoints/best-epoch=00-val_loss=1.954.ckpt' as top 1


Epoch 1: 100%|██████████| 36/36 [00:08<00:00,  4.14it/s, v_num=9, train_loss_step=986.0, train_loss_epoch=1.07e+3]

...

Epoch 499: 100%|██████████| 36/36 [00:08<00:00,  4.15it/s, v_num=9, train_loss_step=38.30, train_loss_epoch=43.20]

Epoch 499, global step 18000: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 36/36 [00:08<00:00,  4.03it/s, v_num=9, train_loss_step=38.30, train_loss_epoch=43.20]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> testing  <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Testing DataLoader 0: 100%|██████████| 12/12 [00:01<00:00,  9.99it/s]


In [15]:
# compute ssp for pred and ture with shape [num,nx,ny,nt]
def get_ssp(pred,true,ax=(1,2)):
    fpred = np.fft.fft2(pred,axes=ax)
    ftrue = np.fft.fft2(true,axes=ax)
    norm_error = np.linalg.norm(fpred-ftrue,axis=ax)
    norm_pred = np.linalg.norm(fpred,axis=ax)
    norm_true = np.linalg.norm(ftrue,axis=ax)
    ssps = norm_error/(norm_pred+norm_true)
    ssp = np.mean(ssps)
    return ssp,ssps

In [ ]:
# show the given frames from an example
inputs = np.load('./work_dirs/convlstm/saved/inputs.npy')
preds = np.load('./work_dirs/convlstm/saved/preds.npy')
trues = np.load('./work_dirs/convlstm/saved/trues.npy')
# [n,16,1,64,64]->[n,64,64,16]
true_u = trues.squeeze(2).transpose(0,2,3,1)
pred_u = preds.squeeze(2).transpose(0,2,3,1)
# mean over axis 1,2,3 while keep axis 0, then normalized with hs
#rmse_e_hs = np.mean(np.sqrt(np.mean((pred_e-true_e)**2,axis=(1,2,3)))/hs)
rmse_u_hs = np.mean(np.sqrt(np.mean((pred_u-true_u)**2,axis=(1,2,3)))/hs)
#mae_e_hs = np.mean(np.mean(np.abs(pred_e-true_e),axis=(1,2,3))/hs)
mae_u_hs = np.mean(np.mean(np.abs(pred_u-true_u),axis=(1,2,3))/hs)
#ssp_e,ssps_e = get_ssp(pred_e,true_e)
ssp_u,ssps_u = get_ssp(pred_u,true_u)
#print('normalized by Hs->rmse_e:%f, rmse_u:%f, mae_e:%f, mae_u:%f, ssp_e:%f, ssp_u:%f' % (rmse_e_hs,rmse_u_hs,mae_e_hs,mae_u_hs,ssp_e,ssp_u)) 
print('normalized by Hs->rmse_u:%f, mae_u:%f, ssp_u:%f' % (rmse_u_hs,mae_u_hs,ssp_u)) 


normalized by Hs->rmse_u:0.202652, mae_u:0.142464, ssp_u:0.264106


: 

The results of ConvLSTM and SimVP-gSTA: 

| Model   | MAE/Hs | RMSE/Hs | SSP |
|---------|-----|------|-----|
| ConvLSTM|  0.142464   | 0.202652     | 0.264106    |
| SimVP   |  0.073016   | 0.116635     | 0.139385    |
